In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from ogusa import utils
import scipy.ndimage.filters as filter

ModuleNotFoundError: No module named 'ogusa'

In [3]:
def get_labor_data():

    # read in "raw" CPS data to calculate moments
    # these data were actually cleaned in hours_data_cps_setup.do
    fileDir = os.path.dirname(os.path.realpath('__file__'))
    filename = os.path.join(
        fileDir,
        '../Data/Current_Population_Survey/cps_est_ability_hours_1992to2013.dta')
    filename = os.path.abspath(os.path.realpath(filename))
    cps = pd.read_stata(filename, columns=['year', 'age', 'hours',
                                           'hours_unit', 'wtsupp'])
    return cps

In [4]:
cps = get_labor_data()

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/GoogleDrive/My Drive/OG-USA/Data/Current_Population_Survey/cps_est_ability_hours_1992to2013.dta'

In [2]:
def compute_labor_moments(cps, S):
    '''
    ------------------------------------------------------------------------
        Compute moments from labor data
    ------------------------------------------------------------------------
    Inputs:
        cps         = pandas DF, raw data from SCF
    Objects created in the function:
        labor_dist_data = [S,] array of labor moments

    Returns:
        labor_dist_data
    ------------------------------------------------------------------------
    '''

    # Find fraction of total time people work on average by age
    cps['hours_wgt'] = cps['hours']*cps['wtsupp']
    columns = ['hours_wgt', 'wgt', 'avg_hours']
    by_age = pd.DataFrame(columns=columns)
    # by_age = by_age.fillna(0) # with 0s rather than NaNs

    by_age['hours_wgt'] = cps.groupby(['age'])['hours_wgt'].sum()
    by_age['wgt'] = cps.groupby(['age'])['wtsupp'].sum()
    by_age['avg_hours'] = by_age['hours_wgt']/by_age['wgt']

    # get fraction of time endowment worked (assume time
    # endowment is 24 hours minus required time to sleep)
    by_age['frac_work'] = by_age['avg_hours']/(365*16.)

    # Data have sufficient obs through age  80
    # Fit a line to the last few years of the average labor
    # participation which extends from ages 76 to 100.
    slope = (by_age['frac_work'][-1] - by_age['frac_work'][-15]) / (15.)
    # intercept = by_age['frac_work'][-1] - slope*len(by_age['frac_work'])
    # extension = slope * (np.linspace(56, 80, 23)) + intercept
    # to_dot = slope * (np.linspace(45, 56, 11)) + intercept

    labor_dist_data = np.zeros(80)
    labor_dist_data[:60] = by_age['frac_work']
    labor_dist_data[60:] = by_age['frac_work'][-1] + slope*range(20)

    # the above computes moments if the model period is a year
    # the following adjusts those moments in case it is smaller
    labor_dist_out = filter.uniform_filter(labor_dist_data,
                                           size=int(80 / S))[::int(80 / S)]

    return labor_dist_out